## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [18]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                              initial_value=tf.random.uniform(shape=[28*28, 100],
                                                              minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                              initial_value=tf.random.uniform(shape=[100, 10],
                                                              minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x=tf.reshape(x,shape=[-1,28*28])
        h1 = tf.tanh(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [10]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [21]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.9044805 ; accuracy 0.59078336
epoch 1 : loss 1.90055 ; accuracy 0.5929
epoch 2 : loss 1.8966235 ; accuracy 0.59491664
epoch 3 : loss 1.8927007 ; accuracy 0.59756666
epoch 4 : loss 1.8887818 ; accuracy 0.5995167
epoch 5 : loss 1.884867 ; accuracy 0.60146666
epoch 6 : loss 1.8809558 ; accuracy 0.60361665
epoch 7 : loss 1.8770489 ; accuracy 0.60538334
epoch 8 : loss 1.8731458 ; accuracy 0.6076
epoch 9 : loss 1.8692468 ; accuracy 0.6094667
epoch 10 : loss 1.8653518 ; accuracy 0.61116666
epoch 11 : loss 1.8614609 ; accuracy 0.61293334
epoch 12 : loss 1.8575742 ; accuracy 0.61443335
epoch 13 : loss 1.8536919 ; accuracy 0.61611664
epoch 14 : loss 1.8498138 ; accuracy 0.6178167
epoch 15 : loss 1.8459399 ; accuracy 0.61965
epoch 16 : loss 1.8420701 ; accuracy 0.62121665
epoch 17 : loss 1.8382051 ; accuracy 0.62285
epoch 18 : loss 1.8343443 ; accuracy 0.62436664
epoch 19 : loss 1.8304877 ; accuracy 0.6260333
epoch 20 : loss 1.826636 ; accuracy 0.62771666
epoch 21 : loss 1.822788